# COMS 4995_002 Deep Learning Assignment 2
Due on Monday, Oct 30, 11:59pm

This assignment can be done in groups of at most 3 students. Everyone must submit on Courseworks individually.

Write down the UNIs of your group (if applicable)

Member 1: Animesh Anant Sharma, aas2325

Member 2: Himanshu Aggarwal, ha2467

Member 3: Name, UNI

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import scipy
import scipy.misc
import glob
import sys
from __future__ import divisio

In [19]:
batch_size=256
num_steps=10001
n_classes=10
dropout = 0.75
LOGDIR = "/tmp/cifar10/q2_2/"

In [3]:
def conv_layer_reg(input, size_in, size_out,name="conv"):
    with tf.name_scope(name):
        w = tf.Variable(tf.truncated_normal([5, 5, size_in, size_out], stddev=0.1), name="W")
        b = tf.Variable(tf.constant(0.1, shape=[size_out]), name="B")
        conv = tf.nn.conv2d(input, w, strides=[1, 1, 1, 1], padding="SAME")
        pre_activation = tf.nn.bias_add(conv, b)
        act = tf.nn.relu(pre_activation)
        tf.summary.histogram("weights", w)
        tf.summary.histogram("biases", b)
        tf.summary.histogram("activations", act)
        return act, tf.nn.l2_loss(w)

In [4]:
def conv_layer_dropout(input, size_in, size_out,name="conv"):
    with tf.name_scope(name):
        w = tf.Variable(tf.truncated_normal([5, 5, size_in, size_out], stddev=0.1), name="W")
        b = tf.Variable(tf.constant(0.1, shape=[size_out]), name="B")
        conv = tf.nn.conv2d(input, w, strides=[1, 1, 1, 1], padding="SAME")
        pre_activation = tf.nn.bias_add(conv, b)
        act = tf.nn.relu(pre_activation)
        tf.summary.histogram("weights", w)
        tf.summary.histogram("biases", b)
        tf.summary.histogram("activations", act)
        return act, tf.nn.l2_loss(w)

In [5]:
def fc_layer_reg(input, size_in, size_out, name="fc"):
    with tf.name_scope(name):
        w = tf.Variable(tf.truncated_normal([size_in, size_out], stddev=0.1), name="W")
        b = tf.Variable(tf.constant(0.1, shape=[size_out]), name="B")
        act = tf.matmul(input, w) + b
        tf.summary.histogram("weights", w)
        tf.summary.histogram("biases", b)
        tf.summary.histogram("activations", act)
        return act, tf.nn.l2_loss(w)

In [6]:
def fc_layer_dropout(input, size_in, size_out, keep_prob, name="fc"):
    with tf.name_scope(name):
        w = tf.Variable(tf.truncated_normal([size_in, size_out], stddev=0.1), name="W")
        b = tf.Variable(tf.constant(0.1, shape=[size_out]), name="B")
        act = tf.matmul(input, w) + b
        act_drop=tf.nn.dropout(act,keep_prob)
        tf.summary.histogram("weights", w)
        tf.summary.histogram("biases", b)
        tf.summary.histogram("activations", act)
        return act_drop, tf.nn.l2_loss(w)

In [7]:
def getBatch(X,y, batchsize):
    indices=np.random.choice(X.shape[0], batchsize, replace=False)
    X_batch=X[indices,:,:,:]
    y_batch=y[:,indices]
    return X_batch,y_batch.T

In [33]:
def cifar10_enhanced(num_conv,num_fc, X_complete, y_complete, X_test):
    
    X_training=X_complete[:45000,:,:,:]
    X_testing=X_complete[49000:,:,:,:]
    Y_training=y_complete[:,:45000]
    Y_testing=y_complete[:,49000:]
    tf.reset_default_graph()
    sess = tf.Session()
    keep_prob = tf.placeholder(tf.float32)
    learning_rate= tf.placeholder(tf.float32)
    x=tf.placeholder(tf.float32,shape=[None,32,32,3],name="x")
    y=tf.placeholder(tf.int32,shape=[None,10],name="labels")
    conv1, l2_conv1 = conv_layer_reg(x, 3, 32, "conv1")
    pool1=tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME",name="pool1")
    norm1 = tf.nn.lrn(pool1, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75,name='norm1')
    regularizer=l2_conv1
    conv_out,l2_conv2 = conv_layer_reg(norm1, 32, 32, "conv2")
    regularizer=regularizer+l2_conv2
    norm2 = tf.nn.lrn(conv_out, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75,
                    name='norm2')
    pool2=tf.nn.max_pool(norm2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME",name="pool2")
    
    flattened=tf.reshape(pool2,[-1,8*8*32])
    
    fc1, l2_fc_1 = fc_layer_dropout(flattened, 8 * 8 * 32, 256, keep_prob, "fc1")
    relu = tf.nn.relu(fc1)
    regularizer=regularizer+l2_fc_1
    embedding_input = relu
    tf.summary.histogram("fc1/relu", relu)
    embedding_size = 256
    logits, l2_fc_2 = fc_layer_reg(relu, 256, 10, "fc2")
    regularizer=regularizer+l2_fc_2
    
    with tf.name_scope("xent"):
        xent = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y)+0.01*regularizer, name="xent")
        tf.summary.scalar("xent", xent)
    
    with tf.name_scope("train"):
            optimizer = tf.train.AdamOptimizer(learning_rate)
            # Get the gradient pairs (Tensor, Variable)
            grads = optimizer.compute_gradients(xent)
            # Update the weights wrt to the gradient
            train_step = optimizer.apply_gradients(grads)
            # Save the grads with tf.summary.histogram
            for index, grad in enumerate(grads):
                tf.summary.histogram("{}-grad".format(grads[index][1].name.replace(':','_')), grads[index])
    
    with tf.name_scope("accuracy"):
        correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        tf.summary.scalar("accuracy", accuracy)
        
    summ = tf.summary.merge_all()
    
    train_writer = tf.summary.FileWriter(LOGDIR + '/train', sess.graph)
    test_writer = tf.summary.FileWriter(LOGDIR + '/test', sess.graph)
    
    sess.run(tf.global_variables_initializer())
    
    rate=0.002
    drop_prob=0.8
    
    
    for i in range(0,num_steps):
        X_batch, Y_batch=getBatch(X_training,Y_training,128)
        
        if i%3000==0:
            rate/=2
        
        if i % 500 == 0:
            [train_accuracy, s, xent_e] = sess.run([accuracy, summ,xent], feed_dict={learning_rate:rate,keep_prob:1,x: X_batch, y: Y_batch})
            train_writer.add_summary(s, i)
            print ("Step {} Train accuracy {} Training Loss {} ").format(i, train_accuracy, xent_e)
            [test_accuracy,sum]= sess.run([accuracy, summ], feed_dict={learning_rate:rate, keep_prob:1,x: X_testing, y: Y_testing.T})
            test_writer.add_summary(sum, i)
            print ("Step {} Validation set accuracy {}").format(i,test_accuracy)
            
        sess.run(train_step,feed_dict={learning_rate:rate,keep_prob:drop_prob,x:X_batch,y:Y_batch})
    
    logits_final=sess.run([logits],feed_dict={learning_rate:rate,keep_prob:1,x:X_test,y:Y_batch})
    print(logits.shape)
    save_predictions("/Users/himanshuaggarwal/Desktop/hw2_output/q2_logits.npy",logits_final.T)

In [9]:
# Helper functions

def get_img_array(path):
    """
    Given path of image, returns it's numpy array
    """
    return scipy.misc.imread(path)

def get_files(folder):
    """
    Given path to folder, returns list of files in it
    """
    filenames = [file for file in glob.glob(folder+'*/*')]
    filenames.sort()
    return filenames

def get_label(filepath, label2id):
    """
    Files are assumed to be labeled as: /path/to/file/999_frog.png
    Returns label for a filepath
    """
    tokens = filepath.split('/')
    label = tokens[-1].split('_')[1][:-4]
    if label in label2id:
        return label2id[label]
    else:
        sys.exit("Invalid label: " + label)

In [10]:
# Functions to load data

def get_labels(folder, label2id):
    """
    Returns vector of labels extracted from filenames of all files in folder
    :param folder: path to data folder
    :param label2id: mapping of text labels to numeric ids. (Eg: automobile -> 0)
    """
    files = get_files(folder)
    y = []
    for f in files:
        y.append(get_label(f,label2id))
    return np.array(y)

def one_hot(y, num_classes=10):
    """
    Converts each label index in y to vector with one_hot encoding
    """
    y_one_hot = np.zeros((y.shape[0], num_classes))
    a=range(0,y.shape[0])
    y_one_hot[a,y] = 1
    return y_one_hot

def get_label_mapping(label_file):
    """
    Returns mappings of label to index and index to label
    The input file has list of labels, each on a separate line.
    """
    with open(label_file, 'r') as f:
        id2label = f.readlines()
        id2label = [l.strip() for l in id2label]
    label2id = {}
    count = 0
    for label in id2label:
        label2id[label] = count
        count += 1
    return id2label, label2id

def get_images(folder):
    """
    returns numpy array of all samples in folder
    each column is a sample resized to 30x30 and flattened
    """
    files = get_files(folder)
    images = []
    count = 0
    
    for f in files:
        count += 1
        if count % 10000 == 0:
            print("Loaded {}/{}".format(count,len(files)))
        img_arr = get_img_array(f)
        img_arr = img_arr.flatten() / 255.0
        images.append(img_arr)
    X = np.column_stack(images)
    return X

def get_train_data(data_root_path):
    """
    Return X and y
    """
    train_data_path = data_root_path + 'train'
    id2label, label2id = get_label_mapping(data_root_path+'labels.txt')
    print(label2id)
    X = get_images(train_data_path)
    y = get_labels(train_data_path, label2id)
    return X, y

def save_predictions(filename, y):
    """
    Dumps y into .npy file
    """
    np.save(filename, y)

In [11]:
# Load the data
data_root_path = '/Users/himanshuaggarwal/Desktop/deep_learning_projects/cifar10-hw1/'
X_train, y_train = get_train_data(data_root_path) # this may take a few minutes
print(X_train.shape)
print(y_train.shape)
X_test = get_images(data_root_path + 'test')
print('Data loading done')

{'horse': 7, 'automobile': 1, 'deer': 4, 'dog': 5, 'frog': 6, 'cat': 3, 'truck': 9, 'ship': 8, 'airplane': 0, 'bird': 2}
Loaded 10000/50000
Loaded 20000/50000
Loaded 30000/50000
Loaded 40000/50000
Loaded 50000/50000
(3072, 50000)
(50000,)
Loaded 10000/10000
Data loading done


In [12]:
X = X_train.astype('float32')
X_test = X_test.astype('float32')
X = X.T


#training set
X_train = X[:45000]
#mean of training dataset
me = np.mean(X_train, axis = 0)
#std of training dataset
st = np.std(X_train, axis = 0)
X -= me
X /= st
y=one_hot(y_train,10)
X= np.reshape(X, [-1, 32, 32, 3])


#test dataset
X_test = X_test.T
#test set normalization
X_test -= me
X_test /= st
X_test= np.reshape(X_test, [-1, 32, 32, 3])
print(X_test.shape)
y=y.T

(10000, 32, 32, 3)


In [15]:
cifar10_enhanced(2,2,X,y,X_test)

Step 0 Train accuracy 0.1015625 Training Loss 29.4365444183 
Step 0 Validation set accuracy 0.119999997318
Step 500 Train accuracy 0.5546875 Training Loss 3.08751392365 
Step 500 Validation set accuracy 0.550999999046
Step 1000 Train accuracy 0.7109375 Training Loss 1.56705629826 
Step 1000 Validation set accuracy 0.652999997139
Step 1500 Train accuracy 0.6015625 Training Loss 1.46230316162 
Step 1500 Validation set accuracy 0.638000011444
Step 2000 Train accuracy 0.671875 Training Loss 1.34932303429 
Step 2000 Validation set accuracy 0.660000026226
Step 2500 Train accuracy 0.6953125 Training Loss 1.29681134224 
Step 2500 Validation set accuracy 0.685000002384
Step 3000 Train accuracy 0.734375 Training Loss 1.15503358841 
Step 3000 Validation set accuracy 0.68599998951
Step 3500 Train accuracy 0.703125 Training Loss 1.21635103226 
Step 3500 Validation set accuracy 0.713999986649
Step 4000 Train accuracy 0.8046875 Training Loss 1.04057741165 
Step 4000 Validation set accuracy 0.68999999

In [16]:
data=np.load("/Users/himanshuaggarwal/Desktop/hw2_output/q2_logits.npy")